In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import matplotlib
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
train = pd.read_csv('../input/train.csv')
# Any results you write to the current directory are saved as output.

In [ ]:
y = train['SalePrice']
x1 = train.drop(['SalePrice'],axis = 1)

In [ ]:
##wait to run
#x = pd.get_dummies(x)
#x = x.fillna(x.mean(),axis = 0)
#x.shape
test1 = pd.read_csv('../input/test.csv')
#test = pd.get_dummies(test)
#test = test.fillna(test.mean())
#test.shape
test1.head()

In [ ]:
all_data = pd.concat((x1.loc[:,'MSSubClass':'SaleCondition'],test1.loc[:,'MSSubClass':'SaleCondition']))
all_data = pd.get_dummies(all_data)
all_data = all_data.fillna(all_data.mean())

x = all_data[:x1.shape[0]]
test = all_data[x1.shape[0]:]



In [ ]:

#first try on xgboost
import xgboost as xgb
from pandas import DataFrame

bst =xgb.XGBRegressor()
bst.fit(x,y,verbose = True)
preds = bst.predict(test)


In [ ]:
from sklearn import metrics
preds_train = bst.predict(x)
metrics.r2_score(y.values,preds_train)

In [ ]:
#tune the hyperparameters
from xgboost import XGBRegressor
from sklearn import cross_validation,metrics
from sklearn.grid_search import GridSearchCV

In [ ]:
#define the valuation
def modelfit(alg,x,y,useTrainCV = True, cv_folds = 5,early_stopping_rounds = 100):
    alg.fit(x,y,eval_metric = 'rmse')
    train_pred = alg.predict(x)
    print ("\n for this model")
    print (metrics.r2_score(y,train_pred))


In [ ]:
#parameters setting
xgb1 = XGBRegressor(base_score = 0.5,colsample_bylevel = 1, colsample_bytree = 1,
                   gamma = 0,
                   learning_rate = 0.1,
                   max_delta_step = 0,
                   max_depth = 3,
                   min_child_weight = 1,
                   missing = None,
                   n_estimator = 100,
                   nthread = 1,
                   objective = 'reg:linear',
                   reg_alpha = 0,
                   reg_lambda = 1,
                   scale_pos_weight = 1,
                   silent = True,
                   subsample = 1)
modelfit(xgb1,x,y)

In [ ]:
#use GridSearchCV to find better model
param_test1 = {
    'max_depth':[3,5,7,9],
    'min_child_weight':[2,4,6,8]
}

gsearch1 = GridSearchCV(estimator = XGBRegressor(base_score = 0.5,
                                                 colsample_bylevel = 1, 
                                                 colsample_bytree = 1,
                                                 gamma = 0,
                                                 learning_rate = 0.1,
                                                max_delta_step = 0,
                   missing = None,
                   n_estimator = 100,
                   nthread = -1,
                   objective = 'reg:linear',
                   reg_alpha = 0,
                   reg_lambda = 1,
                   scale_pos_weight = 1,
                   silent = True,
                   subsample = 1),
                   param_grid = param_test1, scoring = 'r2',cv = 5)
gsearch1.fit(x,y)
                   

In [ ]:
gsearch1.grid_scores_,gsearch1.best_params_,gsearch1.best_score_

In [ ]:
param_test2 = {
    'max_depth':[6,7,8],
    'min_child_weight':[3,4,5]
}

gsearch2 = GridSearchCV(estimator = XGBRegressor(base_score = 0.5,
                                                 colsample_bylevel = 1, 
                                                 colsample_bytree = 1,
                                                 gamma = 0,
                                                 learning_rate = 0.1,
                                                max_delta_step = 0,
                   missing = None,
                   n_estimator = 100,
                   nthread = -1,
                   objective = 'reg:linear',
                   reg_alpha = 0,
                   reg_lambda = 1,
                   scale_pos_weight = 1,
                   silent = True,
                   subsample = 1),
                   param_grid = param_test2, scoring = 'r2',cv = 5)
gsearch2.fit(x,y)

In [ ]:
gsearch2.grid_scores_,gsearch2.best_params_,gsearch2.best_score_

In [ ]:
#parameters setting
xgb2 = XGBRegressor(base_score = 0.5,colsample_bylevel = 0.7, colsample_bytree =0.75,
                    max_depth = 8,
                    min_child_weight = 3,
                   gamma = 0,
                   learning_rate = 0.1,
                   max_delta_step = 0,
                   missing = None,
                   n_estimator = 100,
                   nthread = 1,
                   objective = 'reg:linear',
                   reg_alpha = 0,
                   reg_lambda = 1,
                   scale_pos_weight = 1,
                   silent = True,
                   subsample = 1)
modelfit(xgb2,x,y)

In [ ]:
#parameters setting
xgb3 = XGBRegressor(base_score = 0.5,colsample_bylevel = 0.7, colsample_bytree =0.75,
                    max_depth = 6,
                    min_child_weight = 4,
                   gamma = 0,
                   learning_rate = 0.7,
                   max_delta_step = 0,
                   missing = None,
                   n_estimator = 4000,
                   nthread = -1,
                   objective = 'reg:linear',
                   reg_alpha = 100,
                   reg_lambda = 0.8,
                   scale_pos_weight = 1,
                   silent = True,
                   subsample = 1)
modelfit(xgb3,x,y)
xgb3.fit(x,y)

In [ ]:
xgb_preds = xgb3.predict(test)
#test1.Id.shape
#test.shape
#solution = pd.DataFrame({"id":test1.Id, "SalePrice":preds})
#solution.to_csv("ridge_sol.csv", index = False)

In [ ]:
#Version2 LR

In [ ]:
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score

def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, x, y, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

In [ ]:
model_ridge = Ridge()

In [ ]:
#search for the best regularization parameter
alphas = [0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 30, 50, 75]
cv_ridge = [rmse_cv(Ridge(alpha = alpha)).mean() for alpha in alphas]
import matplotlib.pyplot as plt



%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline

In [ ]:
cv_ridge = pd.Series(cv_ridge,index = alphas)
cv_ridge.plot(title = 'Validation')
plt.xlabel('alpha')
plt.ylabel('rmse')
cv_ridge.min()

In [ ]:
#Lasso
model_lasso = LassoCV(alphas = [1, 0.1, 0.001, 0.0005]).fit(x, y)
rmse_cv(model_lasso).mean()

In [ ]:
coef = pd.Series(model_lasso.coef_, index = x.columns)
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

In [ ]:
imp_coef = pd.concat([coef.sort_values().head(10),
                     coef.sort_values().tail(10)])
imp_coef

In [ ]:
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Coefficients in the Lasso Model")

In [ ]:
matplotlib.rcParams['figure.figsize'] = (6.0, 6.0)

preds = pd.DataFrame({"preds":model_lasso.predict(x), "true":y})
preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals",kind = "scatter")

In [ ]:
lr_preds = model_lasso.predict(test)
preds = 0.7*lr_preds + 0.3*xgb_preds
pred_train = 0.3* model_lasso.predict(x) + 0.7* xgb3.predict(x)
metrics.r2_score(y,pred_train)

In [ ]:
#solution = pd.DataFrame({"id":test1.Id, "SalePrice":preds})
#solution.to_csv("ridge_sol.csv", index = False)

In [ ]:
#V3 add neural network

from keras.layers import Dense
from keras.models import Sequential
from keras.regularizers import l1
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
x_nn = StandardScaler().fit_transform(x)
x_tr,x_val,y_tr,y_val = train_test_split(x_nn,y,random_state =3 )


In [ ]:
model = Sequential()
model.add(Dense(1,input_dim = x.shape[1], W_regularizer=l1(0.001)))

model.compile(loss = 'mse',optimizer = 'adam')

In [ ]:
hist = model.fit(x_tr,y_tr,validation_data = (x_val,y_val))

In [ ]:
nn_preds = model.predict(x_nn)

In [ ]:
pd.Series(model.predict(x_nn)[:,0])

In [ ]:
pred_train = 0.1* model_lasso.predict(x) + 0.8* xgb3.predict(x) + 0.1 * pd.Series(model.predict(x_nn)[:,0])
metrics.r2_score(y,pred_train)


In [ ]:
preds = 0.1* model_lasso.predict(test) + 0.8* xgb3.predict(test) + 0.1 * pd.Series(model.predict(test)[:,0])

In [ ]:
solution = pd.DataFrame({"id":test1.Id, "SalePrice":preds})
solution.to_csv("ridge_sol.csv", index = False)

In [ ]:
#The third combination worked worse than the second version